# Demo MoverActivity

In [1]:
import datetime, time
import simpy

import shapely.geometry

import pandas as pd
import openclsim.core as core
import openclsim.model as model

# setup environment
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)
registry = {}

## Definition of Sites

In [2]:
location_from_site = shapely.geometry.Point(4.18055556, 52.18664444)  # lon, lat
location_to_site = shapely.geometry.Point(4.25222222, 52.11428333)  # lon, lat

Site = type(
    "Site",
    (
        core.Identifiable,
        core.Log,
        core.Locatable,
        core.HasContainer,
        core.HasResource,
    ),
    {},
)

data_from_site = {
    "env": my_env,
    "name": "Winlocatie",
    "ID": "6dbbbdf4-4589-11e9-a501-b469212bff5b",
    "geometry": location_from_site,
    "capacity": 10,
    "level": 2,
}

data_to_site = {
    "env": my_env,
    "name": "Dumplocatie",
    "ID": "6dbbbdf5-4589-11e9-82b2-b469212bff5b",
    "geometry": location_to_site,
    "capacity": 10,
    "level": 0,
}

from_site = Site(**data_from_site)
to_site = Site(**data_to_site)

## Creation of Vessel

In [3]:
TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.Identifiable,
        core.Log,
        core.ContainerDependentMovable,
        core.HasResource,
        core.Processor,
        core.LoadingFunction,
        core.UnloadingFunction,
    ),
    {},
)

def compute_v_provider(v_empty, v_full):
    return lambda x: 10

data_hopper = {
    "env": my_env,
    "name": "Hopper 01",
    "ID": "6dbbbdf6-4589-11e9-95a2-b469212bff5b",
    "geometry": location_from_site,
    "loading_rate": 1,
    "unloading_rate": 1,
    "capacity": 5,
    "compute_v": compute_v_provider(5, 4.5),
}

hopper = TransportProcessingResource(**data_hopper)

## Definition of Move activity

In [4]:
move_activity_data = {
    "env": my_env,
    "name": "Soil movement",
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff5b",
    "registry": registry,
    "mover": hopper,
    "destination": to_site,
}
activity = model.MoveActivity(**move_activity_data)

## Run simulation

In [5]:
my_env.run()

In [6]:
log_df = pd.DataFrame(activity.log)
data =log_df[['Message', 'ActivityState', 'Timestamp', 'Value', 'ActivityID']]
data

,Message,ActivityState,Timestamp,Value,ActivityID
0,move activity Soil movement of Hopper 01 to Du...,START,1970-01-01 00:00:00.000000,-1,6dbbbdf7-4589-11e9-bf3b-b469212bff5b
1,move activity Soil movement of Hopper 01 to Du...,STOP,1970-01-01 00:15:42.824591,-1,6dbbbdf7-4589-11e9-bf3b-b469212bff5b
